In [1]:
%reload_ext autoreload
%autoreload 2

import cv2
import numpy as np
from segmentation import segment_scene_colorful
from utils import show
from params import K, D
from utils import get_roi, get_color_range

In [ ]:
# segment scene
view = "front"
image_file = "data/test_segmentation/0000.jpg"
image = cv2.imread(image_file)
segmentation, (num_red, num_blue) = segment_scene_colorful(image, view)
print(f"Segmented {num_red} red, {num_blue} blue")
show(segmentation)

In [ ]:
# calibrate segmentation
top_image_file = "top.jpg"
top_image = cv2.imread(top_image_file)
top_boxes_roi = get_roi(top_image, "boxes roi")
top_goal_and_stop_line_roi = get_roi(top_image, "goal and stop line roi")

front_image_file = "front.jpg"
front_image = cv2.imread(front_image_file)
front_boxes_roi = get_roi(front_image, "boxes roi")
front_goal_and_stop_line_roi = get_roi(front_image, "goal and stop line roi")

print(f"Boxes roi:")
print(f"    top: x_range, y_range = {top_boxes_roi}")
print(f"    front: x_range, y_range = {front_boxes_roi}")
print(f"Goal and stop line roi:")
print(f"    top: x_range, y_range = {top_goal_and_stop_line_roi}")
print(f"    front: x_range, y_range = {front_goal_and_stop_line_roi}")